In [9]:
"""annotation_parser.py

This module provides utilities to parse annotations from various formats (txt,
line, and XML) and map them to their corresponding images. The primary use
cases include:

1. Parsing annotations from .txt files, especially those in YOLO format.
2. Extracting bounding box annotations from dataset files where each line
represents an image and its associated bounding boxes.
3. Converting XML annotations, typically in PASCAL VOC format, into a
structured format.

Given a directory or file with annotations, the module offers functions to
extract these annotations, adjust any necessary file extensions, and create a
structured Pandas DataFrame. This DataFrame can then be exported to a JSON
format for further processing, analysis, or model training.

The module consolidates the functionality of previously separate utilities into
a single, cohesive class, making it easier to handle various annotation formats
in a unified manner.

Returns:
    DataFrame: A Pandas DataFrame containing the parsed annotations mapped to
    their respective images.

Usage:
    To load the config file into the class:
        parser = AnnotationParser()
        parser.load_config('config.ini')
        parser.process_all_datasets()
"""

import os
import xml.etree.ElementTree as ET
import configparser
import pandas as pd

XFILE_DIR = './data/processed/xml/'
TXT_DIR = './data/processed/txt/'
OUTPUT = './data/processed/annotations.json'
DS_TRAIN = './data/processed/trn'
DS_TEST = './data/processed/test'


class AnnotationParser:
    """
    A utility class for parsing image annotations from various formats.

    The AnnotationParser class provides methods to extract annotations from
    different formats such as .txt (YOLO format), line-based datasets, and XML
    (typically PASCAL VOC format). It consolidates the functionality of
    previously separate utilities into a unified interface, allowing for
    streamlined processing and conversion of annotations into a structured
    Pandas DataFrame. This DataFrame can then be exported to a JSON format
    for further processing or analysis.

    Attributes:
        OUTPUT (str): Default path for the output JSON file.
        FOLDER_IN (str): Default directory containing images and their
          annotations.
        DS3_TRN, DS3_TST, DS4_TRN, DS4_TST (str): Default paths for dataset
          files.
        XFILE_DIR (str): Default directory for XML files.

    Methods:
        load_config(config_file): Load parameters from a configuration file.
        process_all_datasets(): Process all dataset constants and create
        annotation sets for each.
        ... [other methods]

    Usage:
        parser = AnnotationParser()
        parser.load_config('config.ini')
        parser.process_all_datasets()

    Returns:
        DataFrame: A Pandas DataFrame containing the parsed annotations mapped
        to their respective images.
    """

    def process_all_datasets(self):
        """
        Process all dataset constants and create annotation sets for each.

        This function iterates through each of the dataset constants, reads the
        dataset, creates a DataFrame of annotations, and then exports the
        DataFrame to a JSON file.
        """
        datasets = [DS_TRAIN, DS_TEST]
        for dataset in datasets:
            df = self.line_to_dataframe(dataset)
            json_filename = dataset.replace(".txt", ".json").replace(
                "data/processed/", "data/processed/json/"
            )
            df.to_json(json_filename, orient="records", lines=True)
            print(f"Processed {dataset} and saved to {json_filename}")

    def line_to_dataframe(self, dataset):
        """
        Read the dataset file and return a DataFrame with image paths and
        annotations.

        This function reads the specified dataset file, parses each line using
        the parse_line function, and aggregates the parsed data into a
        structured Pandas DataFrame.

        Args:
            dataset (str): Path to the dataset file.

        Returns:
            DataFrame: A Pandas DataFrame containing the image paths and their
            associated bounding box annotations.
        """

        with open(file=dataset, mode="r", encoding="utf-8") as f:
            lines = f.readlines()
        data = [self.from_line(line) for line in lines]
        return pd.DataFrame(data)

    @staticmethod
    def xml_to_annotations(xml_string):
        """Convert a given XML string to the consistent annotation format."""
        root = ET.fromstring(xml_string)
        image_path = root.find('path').text if root.find('path') is not None else root.find('filename').text
        width = float(root.find('size/width').text)
        height = float(root.find('size/height').text)
        annotations = []
        for obj in root.findall('object'):
            annotation = {
                'class': obj.find('name').text,
                'xmin': float(obj.find('bndbox/xmin').text),
                'ymin': float(obj.find('bndbox/ymin').text),
                'xmax': float(obj.find('bndbox/xmax').text),
                'ymax': float(obj.find('bndbox/ymax').text)
            }
            annotations.append(annotation)
        return {
            'image_path': image_path,
            'width': width,
            'height': height,
            'annotations': annotations
        }

    def parse_yolo_line(self, line, image_dir):
        """Parse a YOLO-formatted line and extract the annotation data."""
        parts = line.strip().split()
        image_path = os.path.join(image_dir, parts[0])
        width = 720
        height = 720
        annotations = [{
            'class': 'pothole',
            'xmin': int((float(parts[1]) - float(parts[3])/2) * width),
            'ymin': int((float(parts[2]) - float(parts[4])/2) * height),
            'xmax': int((float(parts[1]) + float(parts[3])/2) * width),
            'ymax': int((float(parts[2]) + float(parts[4])/2) * height)
        }]
        return {
            'image_path': image_path,
            'width': width,
            'height': height,
            'annotations': annotations
        }

    def parse_dataset_line(self, line):
        """Parse a dataset line and extract the annotation data."""
        parts = line.strip().split()
        image_path = parts[0]
        num_boxes = int(parts[1])
        annotations = []
        for i in range(2, 2 + 4 * num_boxes, 4):
            annotation = {
                'class': 'pothole',  # Placeholder, as class is not provided in this format
                'xmin': float(parts[i]),
                'ymin': float(parts[i+1]),
                'xmax': float(parts[i] + parts[i+2]),
                'ymax': float(parts[i+1] + parts[i+3])
            }
            annotations.append(annotation)
        return {
            'image_path': image_path,
            'annotations': annotations
        }

    def process_xml_directory(self, xml_directory):
        """Process all XML files in a directory and return their annotations."""
        all_xml_annotations = []
        for filename in os.listdir(xml_directory):
            if filename.endswith('.xml'):
                xml_file_path = os.path.join(xml_directory, filename)
                with open(file=xml_file_path, mode='r', encoding='utf-8') as f:
                    xml_data = f.read()
                all_xml_annotations.append(self.xml_to_annotations(xml_data))
        return all_xml_annotations

    def process_yolo_directory(self, yolo_directory, image_dir):
        """Process all YOLO-formatted files in a directory and return their annotations."""
        all_yolo_annotations = []
        for yolo_file in os.listdir(yolo_directory):
            if yolo_file.endswith('.txt' or '.csv'):
                yolo_file_path = os.path.join(yolo_directory, yolo_file)
                with open(file=yolo_file_path, mode='r', encoding='utf-8') as f:
                    lines = f.readlines()
                for line in lines:
                    all_yolo_annotations.append(self.parse_yolo_line(line, image_dir))
        return all_yolo_annotations

    def process_all_files(self, xml_directory=XFILE_DIR, yolo_directory=TXT_DIR, image_dir=DS_TRAIN):
        """Process all XML and YOLO-formatted files and save the results to a JSON file."""
        all_xml_annotations = self.process_xml_directory(xml_directory)
        all_yolo_annotations = self.process_yolo_directory(yolo_directory, image_dir)

        # Convert to DataFrame and save to JSON
        df_xml = pd.DataFrame(all_xml_annotations)
        df_yolo = pd.DataFrame(all_yolo_annotations)

        # Combine both DataFrames
        combined_df = pd.concat([df_xml, df_yolo], ignore_index=True)

        # Save to JSON
        combined_df.to_json(OUTPUT, orient='records', lines=True)


In [5]:
"""preprocessor.py
Annotation Preprocessing Utility.

This module provides a utility class for preprocessing image annotations from
various formats. It allows for loading annotations, splitting datasets based on
provided criteria, and combining annotations from different datasets into a
unified format.

Returns:
    DataFrame: A Pandas DataFrame containing the preprocessed annotations.

prep2.py
This module provides a DatasetPreprocessor class to preprocess various datasets
for object detection tasks. The class includes methods to handle different
formats of datasets and convert them into a standardized format suitable for
object detection models. The standardized format includes center coordinates,
width-height format for bounding boxes, normalized coordinates, and mapped
class labels to class IDs.
"""

import os
import pandas as pd
import PIL.Image as Image
import numpy as np


class Preprocessor:
    """
    A utility class for preprocessing image annotations.

    The Preprocessor class provides methods to load annotations from JSON
    files, split datasets based on provided splits, and combine annotations
    from different datasets into a unified format.

    Attributes:
        parser (AnnotationParser): An instance of the AnnotationParser class.
        annotations (dict): A dictionary to store loaded annotations.
    """

    def __init__(self, parser):
        """
        Initialize the Preprocessor with a given parser.

        Args:
            parser (AnnotationParser): An instance of the AnnotationParser
            class.
        """
        self.parser = parser
        self.annotations = {}

    def load_annotations(self, *json_files):
        """
        Load annotations from the provided JSON files into a dictionary.

        Given a list of JSON file paths, this method reads each file and stores
        the annotations in a dictionary with the file name as the key.

        Args:
            *json_files (str): Paths to the JSON files containing annotations.
        """
        for file in json_files:
            df = pd.read_json(file, lines=True)
            self.annotations[file] = df

    # def split_dataset(self, splits_file):
    #     """
    #     Split the dataset based on the provided splits file.

    #     Given a splits file, this method divides the dataset into training and
    #     validation sets based on the specified split criteria.

    #     Args:
    #         splits_file (str): Path to the JSON file containing split criteria.
    #     """

    #     splits = pd.read_json(splits_file)
    #     train_files = splits[splits["split"] == "train"]["filename"].tolist()
    #     val_files = splits[splits["split"] == "val"]["filename"].tolist()

    #     train_df = self.annotations["df1_annotations.json"][
    #         self.annotations["df1_annotations.json"]["filename"].isin(train_files)
    #     ]
    #     val_df = self.annotations["df1_annotations.json"][
    #         self.annotations["df1_annotations.json"]["filename"].isin(val_files)
    #     ]

    #     self.annotations["df1_train"] = train_df
    #     self.annotations["df1_val"] = val_df

    def preprocess(self):
        """
        Combine annotations from different datasets into a unified format.

        This method aggregates annotations from different datasets into a
        single DataFrame, ensuring a consistent format for further processing
        or analysis.

        Returns:
            DataFrame: A Pandas DataFrame containing the combined annotations.
        """
        all_data = pd.concat(
            [
                df
                for key, df in self.annotations.items()
                if key not in ["df1_annotations.json", "df1_splits.json"]
            ]
        )
        return all_data

    def load_data(self, json_file, img_dir):
        """
        Load image and mask data from the provided JSON file and image
        directory.

        Args:
            json_file (str): Path to the JSON file containing image annotations.
            img_dir (str): Directory containing the images referenced in the
            JSON file.

        Returns:
            tuple: A tuple containing two numpy arrays:
            - images (numpy.ndarray): An array of normalized images of shape
                (num_images, 128, 128, 3).
            - masks (numpy.ndarray): An array of masks corresponding to the
                images, where each mask is of shape (128, 128)
                and contains binary values (0 or 1) indicating the absence or
                presence of an object.
        """
        df = pd.concat(pd.read_json(json_file, lines=True))
        images = []
        masks = []

        def process_row(row):
            img_path = os.path.join(img_dir, row["image_path"])
            img = Image.open(img_path).resize((128, 128))
            img_array = np.array(img) / 255.0  # Normalize
            images.append(img_array)

            mask = np.zeros((img.height, img.width))
            for box in row["boxes"]:
                x, y, i, j = box["x"], box["y"], box["width"], box["height"]
                mask[y : y + j, x : x + i] = 1
            masks.append(mask)

        df.apply(process_row, axis=1)

        return np.array(images), np.array(masks)



# CNN For Identifying Potholes

1. Ensure that all the required scripts, listed below, are in the same directory as the notebook or are accessible via the Python path.
    - [ ] `annotation_parser.py`
    - [ ] `features_old.py`
    - [ ] `feature_extractor.py`
    - [ ] `preprocessor.py`
    - [ ] `train_model.py`
    - [ ] `predict_model.py`
    - [ ] `visualize.py`


2. Ensure that all the required data files, listed below are in the specified directories or adjust the paths in the notebook accordingly.



3. Run each cell in the notebook sequentially.
   1. preprocess the data
   2. extract features
   3. design the model
   4. build and train the model
   5. validate the model
   6. visualize the results


In [10]:
from features.feature_extractor import FeatureExtractor as FEx
from data.preprocessor import Preprocessor
from sklearn.model_selection import train_test_split
import pandas as pd


parser = AnnotationParser()
parser.process_all_files()


FileNotFoundError: [WinError 3] The system cannot find the path specified: './data/processed/xml/'

### 1. **Preprocessing**

This section is dedicated to preparing the data for the model. This involves loading the data, possibly normalizing or augmenting it, and splitting it into training, validation, and test sets. The `AnnotationParser` and `Preprocessor` classes are utilized here to load and preprocess the data.

In [13]:
# # Usage
ANON = "annotations.json"

parser = AnnotationParser()
preprocessor = Preprocessor(parser)
preprocessor.load_annotations(ANON)

all_data = preprocessor.preprocess()

# Splitting into training and temporary set
# which will be further split into validation and test sets
X_temp, X_test, y_temp, y_test = train_test_split(
    all_data.drop('class_id', axis=1),
    all_data['class_id'],
    test_size=0.2,
    random_state=42
)

# Splitting the temporary set into validation and test sets
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(all_data.head())


ValueError: Expected object or value

In [ ]:
# Preprocessing
X_train, y_train, X_val, y_val, X_test, y_test = Preprocessor()

### 2. **Features**

This section typically involves feature extraction or engineering.

For CNNs, the raw pixel values of the images are used as features.

However, if there are any additional features that need to be extracted or engineered,
they would be handled in this section. The `FeatureExtractor` class is used here.


In [ ]:
from skimage.feature import local_binary_pattern, hog
import cv2
import numpy as np


class FeatureExtractor:
    """
    A class to extract various features from an image.

    This class provides methods to extract features like LBP, Canny edges,
    color histogram, HOG, and basic image statistics from a given image. It
    also provides a method to extract all features and concatenate them into a
    single vector.
    """
    def __init__(self, image):
        self.image = cv2.cvtColorTwoPlane(
            image, cv2.COLOR_BGR2GRAY
        )  # Convert to grayscale for some features


    def lbp(self, P=8, R=1):
        """
        Compute Local Binary Pattern (LBP) for the image.

        LBP is a simple yet efficient texture operator which labels the pixels
        of an image by thresholding the neighborhood of each pixel and
        considers the result as a binary number.

        Args:
            P (int, optional): Number of circularly symmetric neighbor set
                points. Defaults to 8.
            R (int, optional): Radius of circle. Defaults to 1.

        Returns:
            numpy.ndarray: LBP image.
        """
        return local_binary_pattern(self.image, P=P, R=R, method="uniform")


    def canny_edge(self, lower_threshold=100, upper_threshold=200):
        """
        Compute Canny edge detection for the image.

        The Canny edge detection operator was developed by John F. Canny in
        1986 and uses a multi-stage algorithm to detect a wide range of edges
        in images.

        Args:
            lower_threshold (int, optional): First threshold for the hysteresis
                procedure. Defaults to 100.
            upper_threshold (int, optional): Second threshold for the
                hysteresis procedure. Defaults to 200.

        Returns:
            numpy.ndarray: Binary image of edges.
        """
        return cv2.Canny(self.image, lower_threshold, upper_threshold)


    def color_histogram(self, bins=8):
        """
        Compute color histogram for the grayscale image.

        A histogram represents the distribution of pixel intensities in an
        image.

        Args:
            bins (int, optional): Number of bins for the histogram. Default is 8

        Returns:
            numpy.ndarray: Flattened histogram array.
        """
        hist = cv2.calcHist([self.image], [0], None, [bins], [0, 256])
        cv2.normalize(hist, hist)
        return hist.flatten()


    def hog_features(self, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
        """
        Compute Histogram of Oriented Gradients (HOG) for the image.

        HOG is a feature descriptor used in object detection.

        Args:
            pixels_per_cell (tuple, optional): Size (in pixels) of a cell. Defaults to (8, 8).
            cells_per_block (tuple, optional): Number of cells in each block. Defaults to (2, 2).

        Returns:
            numpy.ndarray: HOG feature vector.
        """
        return hog(
            self.image,
            pixels_per_cell=pixels_per_cell,
            cells_per_block=cells_per_block,
            visualize=False,
        )


    def image_statistics(self):
        """
        Compute basic image statistics: mean, median, standard deviation.

        These statistics provide basic information about the pixel intensity
        distribution in the image.

        Returns:
            tuple: Mean, median, and standard deviation of the image.
        """
        mean = np.mean(self.image)
        median = np.median(self.image)
        std = np.std(self.image)
        return mean, median, std


    def extract_all(self):
        """
        Extract all features and concatenate them into a single vector.

        This method extracts LBP, Canny edges, color histogram, HOG, and basic
        image statistics and concatenates them to form a single feature vector.

        Returns:
            numpy.ndarray: Concatenated feature vector.
        """
        lbp_hist = np.histogram(self.lbp(), bins=8, range=(0, 256))[0]
        canny_edges = self.canny_edge().flatten()
        color_hist = self.color_histogram()
        hog_feat = self.hog_features()
        stats = self.image_statistics()

        # Concatenate all features into a single vector
        return np.concatenate([lbp_hist, canny_edges, color_hist, hog_feat, stats])


In [ ]:
# Feature Extraction
xtra = FEx.extract_features()

### 3. **Model Design**

This is where the architecture of the CNN model is defined. The `design_model` function from the `train_model` script is used to create the model architecture.

Here, we'll define the architecture of the CNN model.

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import Adam

def design_model():
    """
    Design a convolutional neural network (CNN) for binary image classification.

    This function defines a CNN architecture using Keras Sequential API. The
    model consists of convolutional layers, max-pooling layers, a flatten
    layer, and dense layers. The final layer uses a sigmoid activation function
    for binary classification.

    Returns:
        keras.models.Model: A Keras model with the defined CNN architecture.
    """
    model = Sequential([
        Conv2D(
            32,
            (3, 3),
            activation='relu',
            input_shape=(128, 128, 3)
        ),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

def my_optimizer():
    """
    Define the Adam optimizer for training the model.

    This function specifies the Adam optimizer with a learning rate of 0.0001.

    Returns:
        keras.optimizers.Adam: Adam optimizer with the specified learning rate.
    """
    return Adam(lr=0.0001)


# Model Design
model = design_model()


### 4. **Model Build**

After defining the model architecture, this section is dedicated to compiling the model, setting any callbacks, and training the model using the training data. The `build_model` function from the `train_model` script is used here.


In [ ]:
# Model Build

def build_model(model, X_train, y_train, X_val, y_val): # pylint: disable=C0103
    """
    Compile and train the provided CNN model using the training and validation
    data.

    This function compiles the provided model using binary cross-entropy loss
    and the Adam optimizer. It then trains the model using the provided
    training data and validates it using the validation data.

    Args:
        model (keras.models.Model): The CNN model to be trained.
        X_train (numpy.ndarray): Training images.
        y_train (numpy.ndarray): Training labels.
        X_val (numpy.ndarray): Validation images.
        y_val (numpy.ndarray): Validation labels.

    Returns:
        tuple: A trained Keras model and its training history.
    """
    model.compile(
        optimizer=my_optimizer(),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    history = model.fit(
        X_train,
        y_train,
        epochs=10,
        validation_data=(X_val, y_val)
    )
    return model, history

model, history = build_model(model, X_train, y_train, X_val, y_val)

### 5. **Validation**:

Once the model is trained, it's important to evaluate its performance on a validation or test set to understand how well it's likely to perform on unseen data. The `validate_model` function from the `predict_model` script is used here.

In [ ]:
import numpy as np


def validate_model(model, X_test, y_test): # pylint: disable=C0103
    """
    Validate the performance of a trained model on test data.

    This function evaluates the provided model using the test data and prints
    the accuracy of the model.

    Args:
        model (keras.models.Model): The trained CNN model.
        X_test (numpy.ndarray): Test images.
        y_test (numpy.ndarray): True labels for the test images.

    Returns:
        float: The accuracy of the model on the test data.
    """
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    print(f"Test Loss: {loss:.4f}")
    return accuracy


def make_predictions(model, X_new): # pylint: disable=C0103
    """
    Make predictions on new, unseen data using the trained model.

    This function uses the provided model to make predictions on a batch of new
    images. It returns the predicted labels for each image.

    Args:
        model (keras.models.Model): The trained CNN model.
        X_new (numpy.ndarray): New images for which predictions are to be made.

    Returns:
        numpy.ndarray: Predicted labels for the new images.
    """
    predictions = model.predict(X_new)
    predicted_labels = np.where(predictions > 0.5, 1, 0).flatten()
    return predicted_labels

In [ ]:
# predicting potholes using machine learning
make_predictions()

# Validation
validate_model(model, X_test, y_test)


### 6. **Visualization**

This section seems to be dedicated to visualizing the results and understanding the model's performance in more detail. The `NeuralNetworkVisualizer` class is used here for various visualizations.

In [ ]:
# Visualization
from visualize import NeuralNetworkVisualizer as nnv

nnv.visualize_activation_maps(model, history)
nnv.calculate_auc()
nnv.plot_roc_curve()